In [1]:
import polars as pl
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt

In [5]:
# read in data
defense_df = pl.read_parquet("../data/defense_season_stats.parquet")
games_df = pl.read_parquet("../data/games_full_data.parquet")
qb_df = pl.read_parquet("../data/qb_game_stats.parquet")